In [1]:
import numpy as np
from math import ceil
from itertools import product

#naming
import hashlib
import json
import base64

#storage
import shelve
import zlib

# filesistem
import tempfile
import os

# pretty printing
from pprint import pformat
from IPython.display import Markdown

def get_sub_sim_name(setup):
    val = json.dumps(setup, separators=(',', ':'), sort_keys=True)  # dump to json
    val = hashlib.sha224(val.encode("utf-8")).digest()  # hash to fast compare
    val = base64.urlsafe_b64encode(val).decode("ascii")
    val += ".subsim"
    return val

def get_simulator_name(V_tilde_def):
    val = hashlib.sha224(V_tilde_def.encode("utf-8")).digest()  # hash to fast compare
    val = base64.urlsafe_b64encode(val).decode("ascii")
    val += ".simulator"
    return val   

with open("simulate.c") as simulate_code:
    SIMULATE_HASH = hashlib.sha224(simulate_code.read().encode("utf-8")).hexdigest()

# Setup della simulazione
Vedi http://localhost:8888/lab/tree/report.pdf per vedere le condizioni su di esse

In [2]:
file_name = "./simulation_data/square_hq.wip"

# seed
seed = [42]

# simulation setups
d = list(np.linspace(0.1, 10, 5))
beta_tilde = [300]
eta = [0.001]
# normalized potential
V_tilde = ["if(y - floor(y) < 0.5) return 0; else return -1;"]

# markov chain setup
step_size = [0.4]  # how much can every point change
local_steps = [5]  # how many local steps should be done 
measure_every = [1000] # how much complete pass are done between a measuration
measuration_to_take = [500] # how much measuration are to be taken

# simulator
simulator_files = ["simulate.c", "mtwister/mtwister.c", "mtwister/mtwister.h"]
sources = ["simulate.c", "mtwister.c"]
invocation_string = "{seed} {d} {eta} {N} {local_steps} {measuration_to_take} {measure_every} {step_size}"

La simulazione viene compattata in un dizionario e viene creato il file .wip (se esiste ne vengono semplicemente aggiornati i setup)

In [3]:
simulation = {
    "seed":seed,
    "d":d,
    "beta_tilde":beta_tilde,
    "eta":eta,
    "V_tilde":V_tilde,
    "step_size":step_size,
    "local_steps":local_steps,
    "measure_every":measure_every,
    "measuration_to_take":measuration_to_take
}
with shelve.open(file_name, "c") as wip_file:
    wip_file[".setup"] = simulation
    wip_file[".invocation_string"] = invocation_string

Prima di tutto viene controllato se il simulatore è cambiato. In caso sia i compilati che le simulazioni fatte vengono pulite

In [4]:
with shelve.open(file_name, "w") as wip_file:
    if ".simulate_hash" in wip_file and wip_file[".simulate_hash"] != SIMULATE_HASH:
        print("Simulator hash has changed, deleting old work")
        for entry in wip_file:
            if not entry.startswith("."):
                del wip_file[entry]
    wip_file[".simulate_hash"] = SIMULATE_HASH

Vengono create le diverse sub_simulazioni per tutte le possibili combinazioni, e creato un indice che tiene conto dello stato di ciascuna

In [5]:
sub_simulations = {}
with shelve.open(file_name, "r") as wip_file:  # to check if simulation is already done
    for setups in product(
        seed,
        d, 
        beta_tilde, 
        eta,
        V_tilde, 
        step_size, 
        local_steps, 
        measure_every, 
        measuration_to_take
    ):
        # creating seed, taking 8 bytes from setup hash, that already has seed inside
        sub_seed = hashlib.sha224(json.dumps(setups, separators=(',', ':'), sort_keys=True).encode("ascii")).digest()[:8] 
        sub_seed = int.from_bytes(sub_seed, "little")
        
        # calculating true eta
        N = ceil(setups[2] / setups[3])  # beta_tilde on eta
        true_eta = setups[2] / N # beta_tilde on N
        
        setups = {  # converting to a dict
            'seed':sub_seed,
            'd':setups[1], 
            'N':N, 
            'eta':true_eta, 
            'V_tilde':setups[4],
            'step_size':setups[5], 
            'local_steps':setups[6], 
            'measure_every':setups[7], 
            'measuration_to_take':setups[8]
        }
        sub_sim_name = get_sub_sim_name(setups)
        sub_simulations[sub_sim_name] = {
            "status":"done" if sub_sim_name in wip_file else "todo",  # done if they are there already
            "setup":setups, 
            'simulator_name':get_simulator_name(setups['V_tilde']) 
        }


print(f"Prepped {len(sub_simulations)} simulations...")

with shelve.open(file_name, "w") as wip_file:
    wip_file[".index"] = sub_simulations

Prepped 5 simulations...


## Compilare il programma

 Vengono generati gli header conteneti i vari $\tilde{V}(y)$. Dopo il file "simulate.c" viene compilato contro di essi, il risultato binario viene compattato e salvato

In [6]:
for V_tilde_def in V_tilde:
    simulator_name = get_simulator_name(V_tilde_def)
    with shelve.open(file_name, "r") as wip_file:
        if simulator_name in wip_file:  #esiste già
            print(f"Skipping already compiled '{simulator_name}'")
            continue
    print(f"Compiling '{simulator_name}'")
    with tempfile.TemporaryDirectory() as build_dir:
        # creating V_header
        with open(os.path.join(build_dir, "V_header.c"), "w") as out:
            out.write("#include <math.h>\ninline static double V_tilde(double y){"+V_tilde_def+"}")
        # copying simulator code
        for sim_file in simulator_files:
            !cp {sim_file} {build_dir}
        # compiling sources
        !gcc -std=gnu11 {" ".join(os.path.join(build_dir, sim_file) for sim_file in sources)} -lm -o {os.path.join(build_dir, "simulator")}
        # shelving
        with open(os.path.join(build_dir, "simulator"), "rb") as compiled:
            with shelve.open(file_name, "w") as wip_file:
                wip_file[get_simulator_name(V_tilde_def)] = zlib.compress(compiled.read())

Compiling 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator'


In [7]:
report = "### Content of the wip file:\n"
with shelve.open(file_name, "r") as wip_file:
    for item in wip_file:
        report += f"#### {item}:\n    "
        if item.endswith(".simulator"):
            report += "<binary data>\n"
        elif item.endswith(".subsim"):
            report += pformat(wip_file[item], depth=1).replace("\n", "\n    ") + "\n"
        else:
            report += pformat(wip_file[item], depth=3).replace("\n", "\n    ") + "\n"
Markdown(report)

### Content of the wip file:
#### .setup:
    {'V_tilde': ['if(y - floor(y) < 0.5) return 0; else return -1;'],
     'beta_tilde': [300],
     'd': [0.1, 2.575, 5.05, 7.525, 10.0],
     'eta': [0.001],
     'local_steps': [5],
     'measuration_to_take': [500],
     'measure_every': [1000],
     'seed': [42],
     'step_size': [0.4]}
#### .simulate_hash:
    'd95d2ad1326601dce16c7c311803bf8dfa9a3c0af7b69f1b543e52b3'
#### .invocation_string:
    ('{seed} {d} {eta} {N} {local_steps} {measuration_to_take} {measure_every} '
     '{step_size}')
#### .index:
    {'7vk7Y5wCHrW4KchfrUgfSJGTGKmpaKkeFrptrQ==.subsim': {'setup': {'N': 300000,
                                                                   'V_tilde': 'if(y '
                                                                              '- '
                                                                              'floor(y) '
                                                                              '< '
                                                                              '0.5) '
                                                                              'return '
                                                                              '0; '
                                                                              'else '
                                                                              'return '
                                                                              '-1;',
                                                                   'd': 2.575,
                                                                   'eta': 0.001,
                                                                   'local_steps': 5,
                                                                   'measuration_to_take': 500,
                                                                   'measure_every': 1000,
                                                                   'seed': 13056684252695461855,
                                                                   'step_size': 0.4},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'todo'},
     'DuaW5chkb-fsUMyHFkaWi6gfAFTvKBbJbYshSw==.subsim': {'setup': {'N': 300000,
                                                                   'V_tilde': 'if(y '
                                                                              '- '
                                                                              'floor(y) '
                                                                              '< '
                                                                              '0.5) '
                                                                              'return '
                                                                              '0; '
                                                                              'else '
                                                                              'return '
                                                                              '-1;',
                                                                   'd': 5.05,
                                                                   'eta': 0.001,
                                                                   'local_steps': 5,
                                                                   'measuration_to_take': 500,
                                                                   'measure_every': 1000,
                                                                   'seed': 16380880228242171167,
                                                                   'step_size': 0.4},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'todo'},
     'PQkbwOW9jhhniL7hGJJYnWGGQCssEm5wLfo7Ag==.subsim': {'setup': {'N': 300000,
                                                                   'V_tilde': 'if(y '
                                                                              '- '
                                                                              'floor(y) '
                                                                              '< '
                                                                              '0.5) '
                                                                              'return '
                                                                              '0; '
                                                                              'else '
                                                                              'return '
                                                                              '-1;',
                                                                   'd': 0.1,
                                                                   'eta': 0.001,
                                                                   'local_steps': 5,
                                                                   'measuration_to_take': 500,
                                                                   'measure_every': 1000,
                                                                   'seed': 7607735254091082090,
                                                                   'step_size': 0.4},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'todo'},
     'tiaXUise1orMi25xvpO0nrSyi-8XavklTMeSKg==.subsim': {'setup': {'N': 300000,
                                                                   'V_tilde': 'if(y '
                                                                              '- '
                                                                              'floor(y) '
                                                                              '< '
                                                                              '0.5) '
                                                                              'return '
                                                                              '0; '
                                                                              'else '
                                                                              'return '
                                                                              '-1;',
                                                                   'd': 7.525,
                                                                   'eta': 0.001,
                                                                   'local_steps': 5,
                                                                   'measuration_to_take': 500,
                                                                   'measure_every': 1000,
                                                                   'seed': 17136282433980624087,
                                                                   'step_size': 0.4},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'todo'},
     'xUYb-HXxqectFVqmuD6tzdUJtMCiMljTwX3big==.subsim': {'setup': {'N': 300000,
                                                                   'V_tilde': 'if(y '
                                                                              '- '
                                                                              'floor(y) '
                                                                              '< '
                                                                              '0.5) '
                                                                              'return '
                                                                              '0; '
                                                                              'else '
                                                                              'return '
                                                                              '-1;',
                                                                   'd': 10.0,
                                                                   'eta': 0.001,
                                                                   'local_steps': 5,
                                                                   'measuration_to_take': 500,
                                                                   'measure_every': 1000,
                                                                   'seed': 5997141380644521480,
                                                                   'step_size': 0.4},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'todo'}}
#### i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator:
    <binary data>
